## Workout Analytics
#### 01. Aquisition with Crawling
#### 02. Transformation with Cleansing, Denoising, Imputation and Normalization(using Exercise-Dictionary)
#### 03. Exploring Basically

In [1]:
import ssl
from urllib.request import Request, urlopen
from functools import reduce
from itertools import count

resfetch = []
try:
    ssl._create_default_https_context = ssl._create_unverified_context

    for idx in count(start=1):
        l = len(resfetch)
        resfetch[l:] = list(filter(
            lambda s: s.startswith('#'),
            reduce(
                lambda param, func: func(param),
                [lambda raw: [] if raw is None else raw.splitlines(), lambda strs: [f(s) for f in [str.strip] for s in strs]],
                urlopen(Request('https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/datasets/Week0%02d.md' % idx)).read().decode('utf-8', errors='replace')
            )
        ))

        if len(resfetch) == l:
            break
except Exception as err:
    (lambda e: None)(err)

print(f'Aquisition Raw Data Count: {len(resfetch)}')

Aquisition Raw Data Count: 2307


In [2]:
from pathlib import Path
import os
import pickle

dictfile = os.path.join(Path(os.getcwd()).parent, 'datasets', 'pickles', 'exercise-dict.pkl')
with open(dictfile, 'rb') as file:
    exercisedict = pickle.load(file)

print(f'Loding Exercise Dictionary Pickle[{dictfile}]')

Loding Exercise Dictionary Pickle[/Users/kickscar/DoWork/PycharmProjects/HELLCHANG-PRACTICES/datasets/pickles/exercise-dict.pkl]


In [3]:
import re
from datetime import datetime

restransform = []
exs_dict = exercisedict['exercises']
eqptdials_dict = exercisedict['equipment-dialects']

exercise_rawdata_count = 0
date_rawdata_count = 0

for res in resfetch:
    if res.startswith('####'):
        exercise_rawdata_count = exercise_rawdata_count + 1

        # 1. split exercise name & record
        exname, exrecord = (s.strip() for s in res.replace('####', '').strip().split(':'))

        # 2. normalize exercise name with equipment
        exname = re.sub(r'[\d+\\.]', '', exname).strip()
        pttrmatch = re.compile(r'([a-zA-Z\s\',()\-]+)\s*\[([a-zA-Z\s\',]+)\]').match(exname)
        exname, equipment = (exname, None) if pttrmatch is None else pttrmatch.groups()

        # 2-1. name
        exname = exname.strip().title()
        for exkey in exs_dict:
            exdials = exs_dict[exkey]['dialects']
            if exname in exdials:
                exname = exkey
                break

        # 2-2. equipment
        if equipment:
            equipment = equipment.strip().title()
            for eqptdials_key in eqptdials_dict:
                if equipment in eqptdials_dict[eqptdials_key]:
                    equipment = eqptdials_key
                    break

            exname = f'{exname}[{equipment}]'

        # 3.
        # sval, unit = re.compile(r'(\d+.\d*)\s*(.*)').match(re.sub(r"[\([{})\]\s]", "", val)).groups()

        restransform[len(restransform):] = [(date, exname, *re.split(r'\s+', exrecord.strip()))]
    elif res.startswith('##'):
        date_rawdata_count = date_rawdata_count + 1
        date = datetime.strptime(res.replace('##', '').strip(), '%Y/%m/%d')

print(f'Raw Data Count(Exercises, Days): ({exercise_rawdata_count}, {date_rawdata_count})')
print(f'Exercise Transform Data(Tuple) Count: {len(restransform)}')

Raw Data Count(Exercises, Days): (2036, 271)
Exercise Transform Data(Tuple) Count: 2036


In [4]:
# 03. Exploring1: How Exercise Days?
days = list(set([t[0] for t in restransform]))
print(f'Exercise Days: {len(days)}')

Exercise Days: 271


In [5]:
# 03. Exploring2: Which Exercises?
exs = [t[1] for t in restransform]
dataset_exs = [(ex, exs.count(ex)) for ex in list(set(exs))]

sorted_by_ex = sorted(dataset_exs, key=lambda t: t[0])
for d in sorted_by_ex:
    print(d)

('Arnold Press[Dumbbell]', 16)
('Assisted Dips[Machine]', 43)
('Assisted Pull-Up[Machine]', 57)
('Back Extension', 36)
('Barbell Lunge[Smith Machine]', 1)
('Behind The Neck Press[Smith Machine]', 4)
('Bent Over Lateral Raise[Dumbbell]', 3)
('Bent Over Row[Barbell]', 7)
('Bent Over Row[Dumbbell]', 16)
('Biceps Curls[Barbell]', 1)
('Biceps Curls[Dumbbell]', 2)
('Biceps Curls[EZ Curl Bar]', 1)
('Biceps Curls[Fixed Barbell]', 2)
('Biceps Curls[Machine]', 13)
('Body-Weight Squat', 5)
('Cable Chest Fly[Cable]', 1)
('Cable Cross-Over[Cable]', 5)
('Chest Fly[Pec Deck Machine]', 41)
('Cross Lunges', 1)
('Dead Bug', 2)
('Deadlift[Barbell]', 87)
('Deadlift[Dumbbell]', 8)
('Deadlift[Kettlebell]', 12)
('Decline Bench Press[Barbell]', 10)
('Dips', 18)
('Face Pulls[Cable]', 1)
('Flat Bench Press[Barbell]', 56)
('Flat Bench Press[Dumbbel]', 1)
('Flat Bench Press[Dumbbell]', 2)
('Flat Bench Press[Free]', 1)
('Flat Bench Press[Smith Machine]', 7)
('Flat Chest Press[Barbell]', 1)
('Flat Chest Press[Dumbb

In [6]:
# 03. Exploring3: and How Often?
sorted_by_count = sorted(dataset_exs, key=lambda t: t[1], reverse=True)
for d in sorted_by_count:
     print(d)

('Lying Leg Curls[Machine]', 116)
('Lat Pulldown[Machine]', 101)
('Plank', 100)
('Seated Chest Press[Machine]', 89)
('Deadlift[Barbell]', 87)
('Hip Adduction[Machine]', 81)
('Hip Abduction[Machine]', 79)
('Shoulder Press[Machine]', 77)
('Seated Row[Machine]', 72)
('Side Lateral Raise[Dumbbell]', 65)
('Leg Extension[Machine]', 62)
('Assisted Pull-Up[Machine]', 57)
('Power Leg Press[Machine]', 56)
('Flat Bench Press[Barbell]', 56)
('Long Pull Row[Cable]', 54)
('Running', 45)
('Straight Arm Pulldown[Cable]', 44)
('Assisted Dips[Machine]', 43)
('Chest Fly[Pec Deck Machine]', 41)
('One Leg Deadlift[Kettlebell]', 37)
('Back Extension', 36)
('Incline Bench Press[Machine]', 35)
('Wide Pulldown Rear[Machine]', 35)
('Side-Lying Hip Abduction', 32)
('Incline Bench Press[Smith Machine]', 26)
('Wrist Roller', 22)
('Incline Bench Press[Barbell]', 20)
('Dips', 18)
('Reverse Pec Deck Fly[Pec Deck Machine]', 18)
('Incline Bench Press[Dumbbell]', 17)
("Hanging Leg Raises[Captain'S Chair]", 16)
('Arnold 